In [1]:
import socket
import json
import base64
import time
import os
import threading
numThreadsPerDomain = 3
numObjectsPerThread = 2
harfile_name = "bbc.har"
PIPELINED = False
foldername = harfile_name.split(".")[0]
harfile = open(harfile_name,"r")
har_json = json.load(harfile)
num_requests = len(har_json["log"]["entries"])
domain_dict = {}
act = threading.activeCount()


In [134]:
har_json

{'log': {'creator': {'name': 'WebInspector', 'version': '537.36'},
  'entries': [{'cache': {},
    'connection': '36186',
    'pageref': 'page_1',
    'request': {'bodySize': 0,
     'cookies': [{'expires': None,
       'httpOnly': False,
       'name': 'BBC-UID',
       'secure': False,
       'value': 'b14e66ffa7d3aa082db62bdf8941cd482d4f46cd2ace780e77b6df142a7b10320Mozilla%2F5.0%20%28X11%3B%20Linux%20x86_64%29%20AppleWebKit%2F537.36%20%28KHTML%2C%20like%20Gecko%29%20Chrome%2F60.0.3112.90%20S'},
      {'expires': None,
       'httpOnly': False,
       'name': 'ckns_policy',
       'secure': False,
       'value': '100'},
      {'expires': None,
       'httpOnly': False,
       'name': 'ckns_policy_exp',
       'secure': False,
       'value': '1535918321750'},
      {'expires': None,
       'httpOnly': False,
       'name': 'ckns_orb_cachedfig',
       'secure': False,
       'value': '{"uk":0,"ck":0,"ad":1,"ap":0,"tb":0,"mb":0,"eu":0,"df":0}'},
      {'expires': None,
       'httpOn

In [132]:
len(har_json["log"]["entries"])

52

In [133]:
num_requests

52

In [2]:
def fetch_thread(request_small):
    if not PIPELINED:
        for i in request_small:
#             print(threading.current_thread().name, " Processed url ", i["request"]["url"])
            url = i["request"]["url"]
            prefix = url[0:5]
            if prefix != "http:":
                continue
            url_split = url.split("://")

            content_file_path = foldername
            content_file_path = content_file_path + "/" + url_split[-1]
            if int(i["response"]["content"]["size"])!=0:
                if content_file_path[-1]=="/":
                    content_file_path = content_file_path + "index.html"
                if not os.path.exists(os.path.dirname(content_file_path)):
                    try:
                        os.makedirs(os.path.dirname(content_file_path))
                    except: # Guard against race condition
                        print("Error :Race")
                        exit(1)
                har_req = i["request"]
                http_version = "HTTP" + har_req["httpVersion"][4:]
                request_str= har_req["method"] + " /" + har_req["url"].split("//")[1].split("/",maxsplit=1)[-1]+" "+ http_version + "\n" 
                hostname = (har_req["url"].split("//")[1]).split("/")[0]
                request_str = request_str + "Host"+": " + hostname+"\n"
                for el in har_req["headers"]:
                    if el["name"] not in ("User-Agent","Connection"):continue
                    request_str = request_str + el["name"]+": " + el["value"]+"\n"
                request_str = request_str + "\n"
        #         print("------------------------------")
        #         print(request_str)
        #         print("------------------------------")
                encoded_request = request_str.encode()
                client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM) 
                print("Hostname is ", hostname)
                client_socket.connect((hostname, 80))
                client_socket.send(encoded_request)
                received_bytes = b""
                client_socket.settimeout(0.2)
                while 1:
                    try:
                        msg = client_socket.recv(1024)
                        if(len(msg)==0):
                            break
                        received_bytes = received_bytes + msg
                    except:
                        break

                header = received_bytes.split(b"\r\n\r\n")[0]
                client_socket.close()
                content = received_bytes.split(b"\r\n\r\n")[-1]
                headerfile = open(content_file_path+"_header","wb")
                headerfile.write(header)
                headerfile.close()
                contentfile = open(content_file_path,"wb")
                contentfile.write(content)
                contentfile.close()
#         else:
#             for i in request_small:
                


In [3]:
def dom_thread_fun(reqlist):
    tl = []
    done = 0
    tind = 0
    while len(reqlist) != done:
        if len(tl) == numThreadsPerDomain:
            while 1:
                a = 0
                for th in tl:
                    if th.isAlive():
                        a = 1
                if a==1:
                    continue
                else:
                    tl =[]
                    break
        tempList = reqlist[done:min(done+numObjectsPerThread, len(reqlist))]
        done += len(tempList)
        nt  = threading.Thread(name=threading.current_thread().name+"fetch"+str(tind),target=fetch_thread,args=(tempList,))
        tl.append(nt)
        tind+=1
        nt.start()

In [4]:
start_time = time.time()
for i in har_json["log"]["entries"]:
    domain_name = i["request"]["url"].split("//",maxsplit=1)[-1].split("/")[0].split(".",maxsplit=1)[1]
    if domain_name in domain_dict.keys():
        domain_dict[domain_name].append(i)
    else:
        domain_dict[domain_name] = [i]
list_of_threads = []
for dom in domain_dict.keys():
    t = threading.Thread(name=dom,target=dom_thread_fun,args=(domain_dict[dom],))
    list_of_threads.append(t)
    t.start()
while(threading.activeCount()!=act):
#     print("-")
    pass
elapsed_time = time.time() - start_time
print("Time Elasped is ", elapsed_time)

Hostname is  Hostname is Hostname is static.bbci.co.uk  
www.bbc.comstatic.bbci.co.uk

Hostname is  Hostname is static.bbci.co.uk 
www.bbc.co.uk
Hostname is  static.bbc.co.uk
Hostname is  static.bbci.co.uk
Hostname is  Hostname is Hostname is static.bbc.co.uk  Hostname is Hostname is 
static.bbci.co.uknav.files.bbci.co.uk  

mybbc.files.bbci.co.uknav.files.bbci.co.uk

Hostname is  Hostname is Hostname is  Hostname is  www.bbc.co.ukstatic.bbci.co.uk Hostname is nav.files.bbci.co.uk

nav.files.bbci.co.uk Hostname is 
 static.bbc.co.uk

ichef.bbci.co.uk
Hostname is Hostname is Hostname is   Hostname is  ichef.bbci.co.ukichef.bbci.co.uk search.files.bbci.co.uk

www.bbc.co.uk

Hostname is  ichef.bbci.co.uk
Hostname is  ichef.bbci.co.uk
Hostname is  search.files.bbci.co.uk
Hostname is  search.files.bbci.co.uk
Hostname is  search.files.bbci.co.uk
Hostname is  search.files.bbci.co.uk
Hostname is  static.bbci.co.uk
Hostname is  static.bbci.co.uk
Hostname is  Hostname is ichef.bbci.co.uk static.

In [110]:
import socket
import json
import base64
import time
import os
import threading
numThreadsPerDomain = 3
numObjectsPerThread = 2
harfile_name = "bbc.har"
PIPELINED = False
foldername = harfile_name.split(".")[0]
harfile = open(harfile_name,"r")
har_json = json.load(harfile)
num_requests = len(har_json["log"]["entries"])
domain_dict = {}
act = threading.activeCount()

In [111]:
num_requests

52

In [112]:
request_small = har_json["log"]["entries"]

In [114]:
har_json= json.load(open("bbc.har","r"))
for i in har_json["log"]["entries"]:
    domain_name = i["request"]["url"].split("//",maxsplit=1)[-1].split("/")[0].split(".",maxsplit=1)[1]
    if domain_name in domain_dict.keys():
        domain_dict[domain_name].append(i)
    else:
        domain_dict[domain_name] = [i]
request_small = domain_dict["bbci.co.uk"]
request_small = request_small[0:6]

In [115]:
domain_dict.keys()

dict_keys(['bbc.com', 'bbci.co.uk', 'bbc.co.uk', 'visualwebsiteoptimizer.com', 'files.bbci.co.uk', 'api.bbc.com', 'effectivemeasure.net', 'crwdcntrl.net', 'googletagservices.com'])

In [121]:
len(request_small)

6

In [122]:
        for i in request_small:
#             print(threading.current_thread().name, " Processed url ", i["request"]["url"])
            url = i["request"]["url"]
            prefix = url[0:5]
            if prefix != "http:":
                continue
            url_split = url.split("://")

            content_file_path = foldername
            content_file_path = content_file_path + "/" + url_split[-1]
            if 1==1:
                if content_file_path[-1]=="/":
                    content_file_path = content_file_path + "index.html"
                if not os.path.exists(os.path.dirname(content_file_path)):
                    try:
                        os.makedirs(os.path.dirname(content_file_path))
                    except: # Guard against race condition
                        print("Error :Race")
                        exit(1)
                har_req = i["request"]
                http_version = "HTTP" + har_req["httpVersion"][4:]
                request_str= har_req["method"] + " /" + har_req["url"].split("//")[1].split("/",maxsplit=1)[-1]+" "+ http_version + "\n" 
                hostname = (har_req["url"].split("//")[1]).split("/")[0]
                request_str = request_str + "Host"+": " + hostname+"\n"
                for el in har_req["headers"]:
                    if el["name"] not in ("User-Agent","Connection"):continue
                    request_str = request_str + el["name"]+": " + el["value"]+"\n"
                request_str = request_str + "\r\n"
#                 print("------------------------------")
#                 print(request_str)
#                 print("------------------------------")
                encoded_request = request_str.encode()
                client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM) 
                print("Hostname is ", hostname)
                client_socket.connect((hostname, 80))
                client_socket.send(encoded_request)
                received_bytes = b""
                client_socket.settimeout(0.6)
                while 1:
                    try:
                        msg = client_socket.recv(1024)
                        print(len(msg))
                        if(len(msg)==0):
                            break
                        received_bytes = received_bytes + msg
                    except:
                        break
                spl = received_bytes.split(b"\r\n\r\n")
#                 print(spl)
                header = spl[0]
                client_socket.close()
                content = spl[1]
                headerfile = open(content_file_path+"_header","wb")
                headerfile.write(header)
                headerfile.close()
                contentfile = open(content_file_path,"wb")
#                 print(content)
                contentfile.write(content)
                contentfile.close()


Hostname is  static.bbci.co.uk
1024
344
1024
1024
1024
1024
1024
1024
1024
1024
1024
360
1024
344
1024
344
1024
344
1024
344
1024
344
1024
1024
247
0
Hostname is  static.bbci.co.uk
727
0
Hostname is  static.bbci.co.uk
1024
1024
688
1024
1024
1024
967
0
Hostname is  static.bbci.co.uk
1024
589
0
Hostname is  static.bbci.co.uk
1024
344
1024
344
1024
344
1024
344
1024
344
1024
344
1024
344
1024
337
0
Hostname is  static.bbci.co.uk
1024
57
0


In [123]:
client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM) 
x = 0
for i in request_small:
#             print(threading.current_thread().name, " Processed url ", i["request"]["url"])
    url = i["request"]["url"]
    prefix = url[0:5]
    if prefix != "http:":
        continue
    url_split = url.split("://")

    content_file_path = foldername
    content_file_path = content_file_path + "/" + url_split[-1]
    if 1:
        if content_file_path[-1]=="/":
            content_file_path = content_file_path + "index.html"
        if not os.path.exists(os.path.dirname(content_file_path)):
            try:
                os.makedirs(os.path.dirname(content_file_path))
            except: # Guard against race condition
                print("Error :Race")
                exit(1)
        har_req = i["request"]
        http_version = "HTTP" + har_req["httpVersion"][4:]
        request_str= har_req["method"] + " /" + har_req["url"].split("//")[1].split("/",maxsplit=1)[-1]+" "+ http_version + "\n" 
        hostname = (har_req["url"].split("//")[1]).split("/")[0]
        request_str = request_str + "Host"+": " + hostname+"\n"
        request_str = request_str + "Connection"+": " + "keep-alive"+"\n"
        for el in har_req["headers"]:
            if el["name"] not in ("User-Agent"):continue
            request_str = request_str + el["name"]+": " + el["value"]+"\n"
        request_str = request_str + "\r\n\r\n"
        print("------------------------------")
        print(request_str)
        print("------------------------------")
        encoded_request = request_str.encode()
        print("Hostname is ", hostname)
        if x==0:
            x=1
            client_socket.connect((hostname, 80))
        client_socket.send(encoded_request)

------------------------------
GET /frameworks/requirejs/lib.js HTTP/1.0
Host: static.bbci.co.uk
Connection: keep-alive



------------------------------
Hostname is  static.bbci.co.uk
------------------------------
GET /frameworks/barlesque/3.21.28/orb/4/script/orb/api.min.js HTTP/1.0
Host: static.bbci.co.uk
Connection: keep-alive



------------------------------
Hostname is  static.bbci.co.uk
------------------------------
GET /frameworks/barlesque/3.21.28/orb/4/script/orb/font.min.js HTTP/1.0
Host: static.bbci.co.uk
Connection: keep-alive



------------------------------
Hostname is  static.bbci.co.uk
------------------------------
GET /wwhp/1.117.0/modules/vendor/pre-built/vwo/mvt-snippet.js HTTP/1.0
Host: static.bbci.co.uk
Connection: keep-alive



------------------------------
Hostname is  static.bbci.co.uk
------------------------------
GET /wwhp/1.117.0/modules/vendor/bower/modernizr/modernizr.js HTTP/1.0
Host: static.bbci.co.uk
Connection: keep-alive



--------------------

In [124]:
received_bytes = b""
client_socket.settimeout(0.6)
while 1:
    try:
        msg = client_socket.recv(1024)
        print(len(msg))
        if(len(msg)==0):
            pass
        received_bytes = received_bytes + msg
    except:
        print("eeeee")
        break
spl = received_bytes.split(b"\r\n\r\n")

1024
344
1024
1024
688
1024
1024
1024
1024
8
1024
1024
1024
1024
1024
352
1024
1024
1024
1024
1024
1024
260
732
1024
344
1024
344
1024
1024
1024
972
1024
594
1024
344
1024
1024
1024
1024
1024
352
1024
1024
1024
1024
1024
68
eeeee


In [125]:
len(spl)

7

In [127]:
spl[0]

b'HTTP/1.0 200 OK\r\nExpires: Thu, 21 Sep 2017 09:11:17 GMT\r\nLast-Modified: Fri, 09 Sep 2016 08:25:54 GMT\r\nETag: "4d1a-53c0ee5132880"\r\nCache-Control: max-age=1800\r\nServer: Apache/2.2.25 (Unix)\r\nContent-Type: application/javascript\r\nAccess-Control-Allow-Origin: *\r\nDate: Thu, 21 Sep 2017 08:45:41 GMT\r\nContent-Length: 19738\r\nConnection: keep-alive'

In [128]:
spl[1]

b'/** vim: et:ts=4:sw=4:sts=4\n * @license RequireJS 2.1.14 Copyright (c) 2010-2014, The Dojo Foundation All Rights Reserved.\n * Available via the MIT or new BSD license.\n * see: http://github.com/jrburke/requirejs for details\n *///Not using strict: uneven strict support in browsers, #392, and causes\n//problems with requirejs.exec()/transpiler plugins that may not be strict.\n/*jslint regexp: true, nomen: true, sloppy: true *//*global window, navigator, document, importScripts, setTimeout, opera */var requirejs,require,define;(function(global){function isFunction(e){return ostring.call(e)==="[object Function]"}function isArray(e){return ostring.call(e)==="[object Array]"}function each(e,t){if(e){var n;for(n=0;n<e.length;n+=1)if(e[n]&&t(e[n],n,e))break}}function eachReverse(e,t){if(e){var n;for(n=e.length-1;n>-1;n-=1)if(e[n]&&t(e[n],n,e))break}}function hasProp(e,t){return hasOwn.call(e,t)}function getOwn(e,t){return hasProp(e,t)&&e[t]}function eachProp(e,t){var n;for(n in e)if(hasP

In [126]:
for n in spl:
    print("--------------------")
    print(n[0:min(len(n),150)])

--------------------
b'HTTP/1.0 200 OK\r\nExpires: Thu, 21 Sep 2017 09:11:17 GMT\r\nLast-Modified: Fri, 09 Sep 2016 08:25:54 GMT\r\nETag: "4d1a-53c0ee5132880"\r\nCache-Control: max-'
--------------------
b'/** vim: et:ts=4:sw=4:sts=4\n * @license RequireJS 2.1.14 Copyright (c) 2010-2014, The Dojo Foundation All Rights Reserved.\n * Available via the MIT or'
--------------------
b'/*!\n * Barlesque - ORB and all\n * Copyright (c) 2015 BBC, all rights reserved.\n */\ndefine("orb/api",function(){"use strict";var n={layout:[]},t=(windo'
--------------------
b'/*!\n * Barlesque - ORB and all\n * Copyright (c) 2015 BBC, all rights reserved.\n */\n!function(){!function(){"use strict";function t(t){l.push(t),1==l.l'
--------------------
b'var _vwo_code=(function(){\n    var account_id=236061,\n        settings_tolerance=2000,\n        library_tolerance=2500,\n        use_existing_jquery=fal'
--------------------
b'/*! modernizr 3.3.1 (Custom Build) | MIT *\n * http://modernizr.com/download/?-c

In [109]:
print(received_bytes)

b''


In [ ]:
import socket
import json
import base64
import os
import threading

harfile_name = "bbc.har"
foldername = harfile_name.split(".")[0]
harfile = open(harfile_name,"r")
har_json = json.load(harfile)
num_requests = len(har_json["log"]["entries"])
for i in har_json["log"]["entries"]:
    url = i["request"]["url"]
    prefix = url[0:5]
    # print(prefix)
    if prefix != "http:":
        continue
    url_split = url.split("://")
    
    content_file_path = foldername
    content_file_path = content_file_path + "/" + url_split[-1]
    if int(i["response"]["content"]["size"])!=0:
#         if not "text" in i["response"]["content"].keys():
#             continue
        print(content_file_path)
        if content_file_path[-1]=="/":
            content_file_path = content_file_path + "index.html"
        # print(os.path.dirname(right_part))
        if not os.path.exists(os.path.dirname(content_file_path)):
            try:
                os.makedirs(os.path.dirname(content_file_path))
            except: # Guard against race condition
                print("Error :Race")
                exit(1)
        har_req = i["request"]
        http_version = "HTTP" + har_req["httpVersion"][4:]
        request_str= har_req["method"] + " /" + har_req["url"].split("//")[1].split("/",maxsplit=1)[-1]+" "+ http_version + "\n" 
        hostname = (har_req["url"].split("//")[1]).split("/")[0]
        request_str = request_str + "Host"+": " + hostname+"\n"
        for el in har_req["headers"]:
            if el["name"] not in ("User-Agent","Connection"):continue
            request_str = request_str + el["name"]+": " + el["value"]+"\n"
#         request_str = request_str + "Conncetion"+": " + "Close"+"\n"
#         request_str = request_str + "User-Agent"+": " + har_req["headers"]+"\n"
        request_str = request_str + "\n"
        print("------------------------------")
        print(request_str)
        print("------------------------------")
        encoded_request = request_str.encode()
        client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM) 
        print("Hostname is ", hostname)
        client_socket.connect((hostname, 80))
        client_socket.send(encoded_request)
        received_bytes = b""
        client_socket.settimeout(0.2)
        while 1:
            try:
                msg = client_socket.recv(1024)
                if(len(msg)==0):
                    break
                received_bytes = received_bytes + msg
            except:
                break
                
        header = received_bytes.split(b"\r\n\r\n")[0]
        client_socket.close()
        content = received_bytes.split(b"\r\n\r\n")[-1]
        headerfile = open(content_file_path+"_header","wb")
        headerfile.write(header)
        headerfile.close()
        contentfile = open(content_file_path,"wb")
        contentfile.write(content)
        contentfile.close()
        
        
        


In [ ]:
import socket
import json
import base64
import os
harfile_name = "bbc.har"
foldername = harfile_name.split(".")[0]
harfile = open(harfile_name,"r")
har_json = json.load(harfile)
num_requests = len(har_json["log"]["entries"])

  

In [ ]:
c = har_json["log"]["entries"][11]

In [ ]:
c

In [ ]:
c

In [ ]:
r = c["request"]

In [ ]:
r

In [ ]:
requ = r["method"] + " /" + r["url"].split("//")[1].split("/",maxsplit=1)[-1]+" "+ r["httpVersion"] + "\n" 

In [ ]:
requ

In [ ]:
url


In [ ]:
requ = requ[:-9] + "HTTP/1.1\n"

In [ ]:
r["url"]

In [ ]:
hostname = (r["url"].split("//")[1]).split("/")[0]

In [ ]:
hostname

In [ ]:
crauy.split(",",maxsplit=1)

In [ ]:
for el in r["headers"]:
    if el["name"] not in ("User-Agent","Connection"):continue
#     if(el["name"]=="Host"):
#         hostname = el["value"]

    requ = requ + el["name"]+": " + el["value"]+"\n"

In [ ]:
requ = requ + "Host"+": " + hostname+"\n"

In [ ]:
requ

In [ ]:
requ = requ+"\n"

In [ ]:
en = requ.encode()

In [ ]:
hostname

In [ ]:
client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM) 
client_socket.connect((hostname, 80))                               


In [ ]:
client_socket.send(en)


In [ ]:
d = b""

In [ ]:
import re
# substitute sequence of non-ASCII characters with single space
# str = re.sub(r'[^\x00-\x7F]+',' ', str)


In [ ]:
while 1:
    msg = client_socket.recv(1024)
    if len(msg)!=1024:
        d = d + msg
        break
    else:
#         msg = re.sub(r'[^\x00-\x7F]+',' ', msg)
        d= d+msg


In [ ]:
len(d)

In [ ]:
d.split(b"\r\n\r\n")[-1]

In [ ]:
d[:777]

In [ ]:
dec = d.decode("utf8")

In [ ]:
dec

In [ ]:
dec[-100:]

In [ ]:
    msg = client_socket.recv(1024)


In [ ]:
    msg = client_socket.recv(4096)
    d= d+msg
#     if not msg:
#         break
#     else:
# #         msg = re.sub(r'[^\x00-\x7F]+',' ', msg)
#         d= d+msg

In [ ]:
msg

In [ ]:
len(msg)

In [ ]:
type(msg)

In [ ]:
print(msg.decode("utf8"))

In [ ]:
d[-100:]

In [ ]:
for dddd in range(190):
    msg = client_socket.recv(1024)
    if not msg:
        break
    else:
#         msg = re.sub(r'[^\x00-\x7F]+',' ', msg)
        d= d+msg.decode()


In [ ]:
len(d)

In [ ]:
print(d[-1300:])

In [ ]:
len(d)

In [ ]:
d [-300:]

In [ ]:
client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM) 
client_socket.connect((hostname, 80))                               

client_socket.send(en)
msg = client_socket.recv(50000)
client_socket.close()
decod = msg.decode('ascii')
print (decod)

In [ ]:
print(decod[-200:-1])

In [ ]:
len(decod)

In [ ]:

# client_socket.close()

# print (msg.decode('ascii'))


In [ ]:
for i in har_json["log"]["entries"]:
    url = i["request"]["url"]
    prefix = url[0:5]

In [ ]:
i

In [ ]:
import json

In [ ]:
a = open("bbc.har")

In [ ]:
a = json.load(a)

In [ ]:
a["log"].keys()

In [ ]:
a["log"]["pages"]

In [ ]:
len(a["log"]["entries"])

In [ ]:
a["log"]["entries"][0]

In [ ]:
a["log"]["entries"][11]

In [ ]:
a["log"]["entries"][11].keys()

In [ ]:
type(a["log"]["entries"][11]["response"]["content"]["text"])

In [ ]:
 s = a["log"]["entries"][11]["response"]["content"]["text"]

In [ ]:
s.decode('base64')

In [ ]:
import base64
coded_string = s
dec = base64.b64decode(coded_string)

In [ ]:
f = open("out.jpg","wb")

In [ ]:
f.write(dec)

In [ ]:
f.close()